En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.


In [10]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [15]:
!pip install statsmodels

In [16]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [17]:
df = pd.read_csv("data/df_norm_estand.csv", index_col = 0)
df.sample(5)

,age,sex,bmi,children,smoker,region,charges,charges_Sklearn,age_robust,bmi_robust,children_robust
676,55,female,40.81,3,no,southeast,12485.80090,0.313005,0.76,-0.690696,-0.5
5,31,female,25.74,0,no,southeast,3756.62160,0.198909,0.24,0.386322,0.0
171,49,male,30.30,0,no,southwest,8116.68000,0.063459,-0.56,-0.047443,0.0
1227,42,male,37.18,2,no,southeast,7162.01220,NaN,NaN,NaN,NaN
397,21,male,31.02,0,no,southeast,16586.49771,0.135588,-0.08,-0.169439,0.0


### Empezamos con el método OLS

In [20]:
lm = ols('charges_Sklearn ~ age + sex + bmi + children + smoker + region',data = df).fit()

In [21]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
sex,1.0,0.021305,0.021305,0.740933,0.389608
smoker,0.0,0.000000,NaN,NaN,NaN
region,3.0,0.122576,0.040859,1.420956,0.235251
age,1.0,0.074100,0.074100,2.577009,0.108803
bmi,1.0,0.042431,0.042431,1.475630,0.224801
children,1.0,0.020796,0.020796,0.723240,0.395326
Residual,839.0,24.124908,0.028754,NaN,NaN


## Método summary 

In [22]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.397
Date:                Mon, 23 Jan 2023   Prob (F-statistic):              0.203
Time:                        10:08:06   Log-Likelihood:                 305.17
No. Observations:                 847   AIC:                            -594.3
Df Residuals:                     839   BIC:                            -556.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.2258      0.035      6.414      0.000       0.157       0.295
sex[T.male]             0.0107      0.012      0.915      0.360      -0.012       0.034
region[T.northwest]    -0.0296      0.017     -1.775      0.076      -0.062       0.003
region[T.southeast]    -0.0201      0.017     -1.186      0.236      -0.053       0.013
region[T.southwest]    -0.0079      0.017     -0.478      0.633      -0.040       0.025
age                     0.0007      0.000      1.786      0.074   -7.32e-05       0.002
bmi                    -0.0012      0.001     -1.181      0.238      -0.003       0.001
children               -0.0041      0.005     -0.850      0.395      -0.014       0.005
==============================================================================
Omnibus:                      275.313   Durbin-Watson:                   2.094
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              808.665
Skew:                           1.624   Prob(JB):                    2.52e-176
Kurtosis:                       6.515   Cond. No.                         319.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Vamos a ver con efectos aditivos 

In [27]:
lm = ols('charges_Sklearn ~ age_robust * bmi_robust * children_robust * smoker * sex * region',data = df).fit()

In [28]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,0.0,0.000000,NaN,NaN,NaN
sex,1.0,0.021305,0.021305,1.198690,0.273920
region,3.0,0.122576,0.040859,2.298839,0.076135
smoker:sex,0.0,0.000000,NaN,NaN,NaN
smoker:region,0.0,0.000000,NaN,NaN,NaN
...,...,...,...,...,...
age_robust:bmi_robust:children_robust:smoker:sex,0.0,0.000000,NaN,NaN,NaN
age_robust:bmi_robust:children_robust:smoker:region,0.0,0.000000,NaN,NaN,NaN
age_robust:bmi_robust:children_robust:sex:region,3.0,0.032697,0.010899,0.613210,0.606562
age_robust:bmi_robust:children_robust:smoker:sex:region,0.0,0.000000,NaN,NaN,NaN


In [29]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     9.368
Date:                Mon, 23 Jan 2023   Prob (F-statistic):           6.03e-61
Time:                        10:27:00   Log-Likelihood:                 538.15
No. Observations:                 847   AIC:                            -948.3
Df Residuals:                     783   BIC:                            -644.8
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 0.1939      0.014     14.123      0.000       0.167       0.221
sex[T.male]                                                               0.0344      0.019      1.796      0.073      -0.003       0.072
region[T.northwest]                                                       0.0199      0.020      1.004      0.316      -0.019       0.059
region[T.southeast]                                                       0.0013      0.019      0.071      0.943      -0.036       0.038
region[T.southwest]                                                       0.0186      0.019      0.990      0.323      -0.018       0.055
sex[T.male]:region[T.northwest]                                          -0.0410      0.028     -1.452      0.147      -0.096       0.014
sex[T.male]:region[T.southeast]                                          -0.0021      0.027     -0.079      0.937      -0.055       0.051
sex[T.male]:region[T.southwest]                                          -0.0511      0.027     -1.885      0.060      -0.104       0.002
age_robust                                                                0.2099      0.027      7.712      0.000       0.156       0.263
age_robust:sex[T.male]                                                   -0.0168      0.036     -0.465      0.642      -0.088       0.054
age_robust:region[T.northwest]                                           -0.0084      0.036     -0.232      0.816      -0.080       0.063
age_robust:region[T.southeast]                                           -0.0495      0.036     -1.378      0.169      -0.120       0.021
age_robust:region[T.southwest]                                           -0.0331      0.038     -0.883      0.377      -0.107       0.041
age_robust:sex[T.male]:region[T.northwest]                               -0.0019      0.052     -0.037      0.971      -0.104       0.100
age_robust:sex[T.male]:region[T.southeast]                                0.0593      0.050      1.184      0.237      -0.039       0.158
age_robust:sex[T.male]:region[T.southwest]                                0.0032      0.051      0.062      0.950      -0.098       0.104
bmi_robust                                                               -0.0044      0.018     -0.238      0.812      -0.041       0.032
bmi_robust:sex[T.male]                                                    0.0100      0.025      0.393      0.695      -0.040       0.060
bmi_robust:region[T.northwest]                                           -0.0160      0.026     -0.614      0.539      -0.067       0.035
bmi_robust:region[T.southeast]                 

## Vemos con efectos no aditivos pero con el Robust

In [23]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'charges_Sklearn', 'age_robust', 'bmi_robust', 'children_robust'],
      dtype='object')

In [24]:
lm = ols('charges_Sklearn ~ age_robust + bmi_robust + children_robust + smoker + sex + region',data = df).fit()

In [25]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,0.0,0.000000,NaN,NaN,NaN
sex,1.0,0.021305,0.021305,1.224060,2.688825e-01
region,3.0,0.122576,0.040859,2.347492,7.134969e-02
age_robust,1.0,9.373890,9.373890,538.567053,2.010992e-92
bmi_robust,1.0,0.001141,0.001141,0.065565,7.979692e-01
children_robust,1.0,0.284207,0.284207,16.328810,5.813903e-05
Residual,839.0,14.602998,0.017405,NaN,NaN


In [26]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     80.46
Date:                Mon, 23 Jan 2023   Prob (F-statistic):           3.03e-89
Time:                        10:19:10   Log-Likelihood:                 517.77
No. Observations:                 847   AIC:                            -1020.
Df Residuals:                     839   BIC:                            -981.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.2074      0.010     20.212      0.000       0.187       0.228
sex[T.male]             0.0107      0.009      1.177      0.240      -0.007       0.029
region[T.northwest]    -0.0023      0.013     -0.179      0.858      -0.028       0.023
region[T.southeast]    -0.0045      0.013     -0.351      0.726      -0.030       0.021
region[T.southwest]    -0.0067      0.013     -0.518      0.604      -0.032       0.019
age_robust              0.1882      0.008     22.611      0.000       0.172       0.205
bmi_robust              0.0010      0.006      0.162      0.871      -0.011       0.013
children_robust         0.0302      0.007      4.041      0.000       0.016       0.045
==============================================================================
Omnibus:                      555.104   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4109.927
Skew:                           3.108   Prob(JB):                         0.00
Kurtosis:                      11.822   Cond. No.                         5.44
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Intepretación

Con el coeficiente podemos ver que todas nuestras variables tienen una relación postiva, menos las de 'region', que es negativa.

El std error 

El r-squared es el porcentaje de la variacion de la variable respuesta. ES decir, nuestras variables predictoras explican el 40,2% de la variacion en los precios del seguro medico (charges). ES un valor bajo.